In [5]:
import torch
import os
from typing import List

import cv2
import torch
from src.data_loader.joints import Joints
from src.utils import read_json
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import pandas as pd
from src.visualization.visualize import plot_hand
from src.data_loader.joints import Joints
from src.data_loader.utils import convert_to_2_5D
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive,GridspecLayout, interact
from tqdm.notebook import tqdm

In [6]:
from src.data_loader.interhand_loader import IH_DB
from src.data_loader.data_set import Data_Set
from src.constants import MASTER_THESIS_DIR, FREIHAND_DATA
from easydict import EasyDict as edict
from torchvision import transforms

In [7]:
train_param = edict(
    read_json(f"{MASTER_THESIS_DIR}/src/experiments/config/training_config.json")
)
train_data = Data_Set(
    config=train_param,
    transform=transforms.ToTensor(),
    train_set=True,
    experiment_type="supervised",
   source =  'interhand'
)
# data = IH_DB("/home/aneesh/Documents/master_thesis/data/raw/InterHand","train")
# train_data =data

## Check number of joints out of the frame.

In [10]:
indices = []
for i in tqdm(range(len(train_data))):
    sample = data[i]
    j  = sample['joints3D']
    k = sample['K']
    if (((j@k.T)/(j@k.T)[:,-1:])[:,:-1]>700).any():
        indices.append(i)
print(len(indices))
# len(indices)/len(data)

## Visualize data.(from data_set class)

In [9]:
@interact(
    idx=widgets.IntSlider(min=0, max=369270, step=1, value=264))
def visualize(idx):
    sample = train_data[idx]
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(131)
    plt.imshow(transforms.ToPILImage()(sample["image"]))
    ax.set_title("Image")
    plot_hand(ax,sample['joints'])
    ax2 = fig.add_subplot(132,projection="3d")
    plot_hand(ax2, sample['joints3D'],plot_3d=True, )
    ax2.set_title("Ground Truth")
    ax3 = fig.add_subplot(133,projection="3d")
    plot_hand(ax3, sample['joints3D_recreated'],plot_3d=True, )
    plot_hand(ax3, sample['joints3D'],plot_3d=True,alpha=0.5, linestyle=":" )
    ax3.set_title("Recreated")
#     print(sample)
    plt.show()

interactive(children=(IntSlider(value=264, description='idx', max=369270), Output()), _dom_classes=('widget-in…

In [11]:
# @interact(
#     idx=widgets.IntSlider(min=0, max=369270, step=1, value=264))
# def visualize(idx):
#     sample = train_data[idx]
# #     sample['joints3D'][:,-1]+=1e-6
#     joints25,_ = convert_to_2_5D(sample['K'], sample['joints3D'])
#     if torch.isnan(joints25).any():
#         print(joints25, sample['K'], sample['joints3D'])
# #     img =  torch.flip(torch.from_numpy(sample["image"]), (0,))
#     fig = plt.figure(figsize=(20,20))
#     ax = fig.add_subplot(131)
# #     plt.imshow(transforms.ToPILImage()(img))
# #     print(sample['joints'])
#     plt.imshow(sample['image'])
#     ax.set_title("Image")
#     j  = sample['joints3D']
#     k = sample['K']
#     j = (j@k.T)/(j@k.T)[:,-1:]
# #     plot_hand(ax, j)
#     plot_hand(ax,joints25)
# #     plot_hand(ax,sample['joints'])
#     ax2 = fig.add_subplot(132,projection="3d")
#     plot_hand(ax2, train_data[idx]['joints3Dworld'],plot_3d=True, )
#     ax2.set_title("Joints world frame")
#     ax3 = fig.add_subplot(133,projection="3d")
#     plot_hand(ax3, train_data[idx]['joints3D'],plot_3d=True, )
#     ax3.set_title("Joints camera frame")
    
# #     plt.savefig("interhand_correct_annotations.png",bbox_inches='tight')
#     plt.show()
# #     print(train_data[idx]['joints3Dworld'])
# #     print(j - joints25)